In [1]:
from torchtext import data,datasets
from torchtext.vocab import GloVe,FastText,CharNGram
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torch
from torchtext.datasets.imdb import IMDB
import sys

In [2]:
torch.__version__
sys.version

'3.7.1 | packaged by conda-forge | (default, Nov 13 2018, 18:33:04) \n[GCC 7.3.0]'

In [3]:
sys.getdefaultencoding()

'utf-8'

In [4]:
is_cuda = False

if torch.cuda.is_available():
    is_cuda=True

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
TEXT = data.Field(lower=True, batch_first=True,fix_length=40,)
LABEL = data.Field(sequential=False,)

In [6]:
train, test = IMDB.splits(TEXT, LABEL)

In [7]:
type(train)

torchtext.datasets.imdb.IMDB

In [8]:
print('train.fields', train.fields)
print('len(train)', len(train))
print('vars(train[0])', vars(train[0]))



train.fields {'text': <torchtext.data.field.Field object at 0x7fd027afc0f0>, 'label': <torchtext.data.field.Field object at 0x7fd027afc160>}
len(train) 25000
vars(train[0]) {'text': ['at', 'first', 'i', "didn't", 'think', 'that', 'ben', 'affleck', 'could', 'really', 'pull', 'off', 'a', 'funny', 'christmas', 'movie,,', 'boy', 'was', 'i', 'wrong,', 'my', 'daughter', 'invited', 'me', 'to', 'watch', 'this', 'with', 'her', 'and', 'i', 'was', 'not', 'disappointed', 'at', 'all.', 'james', 'gandolfini', 'was', 'funny,,', 'i', 'really', 'liked', 'christina', 'appelagate,', 'and', 'catherine', "o'", 'hara', 'was', 'good', 'too,', 'the', 'storyline', 'is', 'what', 'really', 'sold', 'me,,', 'i', 'mean,,', 'too', 'put', 'up', 'with', 'family,,', 'at', 'the', 'table', 'for', 'people', 'you', 'only', 'hardly', 'see', 'but', 'once', 'or', 'twice', 'a', 'year,,', 'and', 'probably', "don't", 'get', 'along', 'with', 'anyway,,', 'you', 'really', 'do', 'need', 'as', 'much', 'alcohol', 'as', "you're", 'syst

In [9]:
TEXT.build_vocab(train, vectors=GloVe(name='6B', dim=300),max_size=10000,min_freq=10)
LABEL.build_vocab(train,)

In [10]:
LABEL.vocab.freqs

Counter({'pos': 12500, 'neg': 12500})

In [11]:
print('train.fields', train.fields)
print('len(train)', len(train))
print('vars(train[0])', vars(train[0]))

train.fields {'text': <torchtext.data.field.Field object at 0x7fd027afc0f0>, 'label': <torchtext.data.field.Field object at 0x7fd027afc160>}
len(train) 25000
vars(train[0]) {'text': ['at', 'first', 'i', "didn't", 'think', 'that', 'ben', 'affleck', 'could', 'really', 'pull', 'off', 'a', 'funny', 'christmas', 'movie,,', 'boy', 'was', 'i', 'wrong,', 'my', 'daughter', 'invited', 'me', 'to', 'watch', 'this', 'with', 'her', 'and', 'i', 'was', 'not', 'disappointed', 'at', 'all.', 'james', 'gandolfini', 'was', 'funny,,', 'i', 'really', 'liked', 'christina', 'appelagate,', 'and', 'catherine', "o'", 'hara', 'was', 'good', 'too,', 'the', 'storyline', 'is', 'what', 'really', 'sold', 'me,,', 'i', 'mean,,', 'too', 'put', 'up', 'with', 'family,,', 'at', 'the', 'table', 'for', 'people', 'you', 'only', 'hardly', 'see', 'but', 'once', 'or', 'twice', 'a', 'year,,', 'and', 'probably', "don't", 'get', 'along', 'with', 'anyway,,', 'you', 'really', 'do', 'need', 'as', 'much', 'alcohol', 'as', "you're", 'syst

In [12]:
d = vars(TEXT.vocab)

In [13]:
d.keys()

dict_keys(['freqs', 'itos', 'stoi', 'vectors'])

In [14]:
TEXT.vocab.vectors



tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0466,  0.2132, -0.0074,  ...,  0.0091, -0.2099,  0.0539],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.7724, -0.1800,  0.2072,  ...,  0.6736,  0.2263, -0.2919],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [15]:
len(TEXT.vocab.stoi)

10002

In [16]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=32, device=-1)

train_iter.repeat = False
test_iter.repeat = False

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [17]:
class EmbNet(nn.Module):
    def __init__(self,emb_size,hidden_size1,hidden_size2=400):
        super().__init__()
        self.embedding = nn.Embedding(emb_size,hidden_size1)
        self.fc = nn.Linear(hidden_size2,3)
        
    def forward(self,x):
        embeds = self.embedding(x).view(x.size(0),-1)
        out = self.fc(embeds)
        return F.log_softmax(out,dim=-1)
        
        

In [18]:
model = EmbNet(len(TEXT.vocab.stoi),10)
model = model.to(device)

In [19]:

optimizer = optim.Adam(model.parameters(),lr=0.001)

In [20]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=32, device=device,shuffle=True)
train_iter.repeat = False
test_iter.repeat = False

In [21]:
def fit(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , batch in enumerate(data_loader):
        text , target = batch.text , batch.label
        if is_cuda:
            text,target = text.to(device),target.to(device)
        
        if phase == 'training':
            optimizer.zero_grad()
        output = model(text)
        loss = F.nll_loss(output,target)
        
        running_loss += F.nll_loss(output,target,size_average=False).item()
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = float(running_loss/len(data_loader.dataset))
    accuracy = float(100. * running_correct/len(data_loader.dataset))
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss,accuracy

In [22]:
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]

In [23]:
train_losses , train_accuracy = [],[]
val_losses , val_accuracy = [],[]

for epoch in range(1,10):

    epoch_loss, epoch_accuracy = fit(epoch,model,train_iter,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model,test_iter,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

/home/walle/miniconda3/envs/pytorch1_py37/lib/python3.7/site-packages/torch/nn/_reduction.py:49: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


training loss is  0.74 and training accuracy is 12964/25000      51.0
validation loss is  0.71 and validation accuracy is 13324/25000      53.0
training loss is  0.68 and training accuracy is 14476/25000      57.0
validation loss is  0.68 and validation accuracy is 14487/25000      57.0
training loss is  0.64 and training accuracy is 15819/25000      63.0
validation loss is  0.65 and validation accuracy is 15627/25000      62.0
training loss is  0.59 and training accuracy is 16929/25000      67.0
validation loss is  0.63 and validation accuracy is 16332/25000      65.0
training loss is  0.55 and training accuracy is 17867/25000      71.0
validation loss is  0.61 and validation accuracy is 16805/25000      67.0
training loss is  0.52 and training accuracy is 18505/25000      74.0
validation loss is   0.6 and validation accuracy is 17160/25000      68.0
training loss is  0.48 and training accuracy is 19080/25000      76.0
validation loss is   0.6 and validation accuracy is 17412/25000   

## Using pretrained Glove word embeddings

In [24]:
TEXT = data.Field(lower=True, batch_first=True,fix_length=40,)
LABEL = data.Field(sequential=False,)

train, test = IMDB.splits(TEXT, LABEL)

TEXT.build_vocab(train,test, vectors=GloVe(name='6B', dim=300),max_size=10000,min_freq=10)
LABEL.build_vocab(train,)

In [25]:
class EmbNet(nn.Module):
    def __init__(self,emb_size,hidden_size1,hidden_size2=400):
        super().__init__()
        self.embedding = nn.Embedding(emb_size,hidden_size1)
        self.fc1 = nn.Linear(hidden_size2,3)

        
    def forward(self,x):
        embeds = self.embedding(x).view(x.size(0),-1)
        out = self.fc1(embeds)
        return F.log_softmax(out,dim=-1)

In [26]:
model = EmbNet(len(TEXT.vocab.stoi),300,12000)
model = model.to(device)

In [27]:
model.embedding.weight.data = TEXT.vocab.vectors.to(device)

In [28]:
model.embedding.weight.requires_grad = False

In [29]:
#optimizer = optim.SGD(model.parameters(),lr=0.001)
optimizer = optim.Adam([ param for param in model.parameters() if param.requires_grad == True],lr=0.001)

In [30]:
train_iter, test_iter = data.BucketIterator.splits((train, test), batch_size=64, device=-1,shuffle=True)
train_iter.repeat = False
test_iter.repeat = False

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


In [31]:
def fit(epoch,model,data_loader,phase='training',volatile=False):
    if phase == 'training':
        model.train()
    if phase == 'validation':
        model.eval()
        volatile=True
    running_loss = 0.0
    running_correct = 0
    for batch_idx , batch in enumerate(data_loader):
        text , target = batch.text , batch.label
        if is_cuda:
            text,target = text.to(device),target.to(device)
        
        if phase == 'training':
            optimizer.zero_grad()
        output = model(text)
        loss = F.nll_loss(output,target)
        
        running_loss += F.nll_loss(output,target,size_average=False).item()
        preds = output.data.max(dim=1,keepdim=True)[1]
        running_correct += preds.eq(target.data.view_as(preds)).cpu().sum()
        if phase == 'training':
            loss.backward()
            optimizer.step()
    
    loss = float(running_loss/len(data_loader.dataset))
    accuracy = float(100. * running_correct/len(data_loader.dataset))
    
    print(f'{phase} loss is {loss:{5}.{2}} and {phase} accuracy is {running_correct}/{len(data_loader.dataset)}{accuracy:{10}.{4}}')
    return loss,accuracy

In [32]:
for epoch in range(1,10):

    epoch_loss, epoch_accuracy = fit(epoch,model,train_iter,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model,test_iter,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

training loss is  0.65 and training accuracy is 15797/25000      63.0
validation loss is  0.68 and validation accuracy is 16044/25000      64.0
training loss is  0.56 and training accuracy is 17743/25000      70.0
validation loss is  0.65 and validation accuracy is 16499/25000      65.0
training loss is  0.53 and training accuracy is 18355/25000      73.0
validation loss is  0.67 and validation accuracy is 16475/25000      65.0
training loss is  0.51 and training accuracy is 18761/25000      75.0
validation loss is  0.71 and validation accuracy is 16269/25000      65.0
training loss is  0.49 and training accuracy is 19036/25000      76.0
validation loss is  0.72 and validation accuracy is 16361/25000      65.0
training loss is  0.48 and training accuracy is 19249/25000      76.0
validation loss is  0.73 and validation accuracy is 16188/25000      64.0
training loss is  0.47 and training accuracy is 19384/25000      77.0
validation loss is  0.75 and validation accuracy is 16263/25000   

In [33]:
for epoch in range(1,10):

    epoch_loss, epoch_accuracy = fit(epoch,model,train_iter,phase='training')
    val_epoch_loss , val_epoch_accuracy = fit(epoch,model,test_iter,phase='validation')
    train_losses.append(epoch_loss)
    train_accuracy.append(epoch_accuracy)
    val_losses.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

training loss is  0.45 and training accuracy is 19730/25000      78.0
validation loss is   0.8 and validation accuracy is 16167/25000      64.0
training loss is  0.44 and training accuracy is 19793/25000      79.0
validation loss is  0.83 and validation accuracy is 16130/25000      64.0
training loss is  0.44 and training accuracy is 19853/25000      79.0
validation loss is  0.84 and validation accuracy is 16004/25000      64.0
training loss is  0.43 and training accuracy is 19944/25000      79.0
validation loss is  0.86 and validation accuracy is 16052/25000      64.0
training loss is  0.43 and training accuracy is 20063/25000      80.0
validation loss is  0.86 and validation accuracy is 16088/25000      64.0
training loss is  0.42 and training accuracy is 20104/25000      80.0
validation loss is  0.88 and validation accuracy is 16038/25000      64.0
training loss is  0.42 and training accuracy is 20198/25000      80.0
validation loss is  0.88 and validation accuracy is 16019/25000   